In [1]:
import uuid
class BinaryTree:
    def __init__(self, data, parent=None):
        self.data = data
        self.parent = parent
        self.uuid= uuid.uuid4()
        self.left = None
        self.right = None    
            
    def addLeftChild(self, data): 
        n = self.__class__(data, self)
        self.left = n
        return n
        
    def addRightChild(self, data):
        n = self.__class__(data, self)
        self.right = n
        return n
        
    def hasLeftChild(self):
        return self.left is not None

    def hasRightChild(self):
        return self.right is not None

    def hasAnyChild(self):
        return self.hasRightChild() or self.hasLeftChild()

    def hasBothChildren(self):
        return self.hasRightChild() and self.hasLeftChild()
    
    def hasNoChildren(self):
        return not self.hasRightChild() and not self.hasLeftChild()
    
    def isLeftChild(self):
        return self.parent and self.parent.left == self

    def isRightChild(self):
        return self.parent and self.parent.right == self

    def isRoot(self):
        return not self.parent

    def isLeaf(self):
        return not (self.right or self.left)
    
            
    def preorder(self):
        if self.isLeftChild():
            yield (self.parent, self, "left")
        elif self.isRightChild():
            yield (self.parent, self, "right")
        if self.hasLeftChild():
            for v in self.left.preorder():
                yield v
        if self.hasRightChild():
            for v in self.right.preorder():
                yield v


In [14]:
class BinarySearchTree(BinaryTree):
        
    def __init__(self, data, parent=None):
        super().__init__(data, parent)
        self.count = 1

    def _insert_hook(self):
        pass
            
    def insert(self, data):
        if data < self.data:
            if self.hasLeftChild():
                self.left.insert(data)
            else:
                self.addLeftChild(data)
                self._insert_hook()
        elif data > self.data:
            if self.hasRightChild():
                self.right.insert(data)
            else:
                self.addRightChild(data)
                self._insert_hook()
        else: #duplicate value
            self.count += 1
            self._insert_hook()
            
    def search(self, data):
        if self.data == data:
            return self
        elif data < self.data and self.left:
            return self.left.search(data)
        elif data > self.data and self.right:
            return self.right.search(data)
        else:
            return None
        
    def delete(self, data):        
        if self.isRoot() and self.hasNoChildren() and self.data==data:#deleting the whole tree
            self.root=None#todo call a destructor that signals GC it can reap
            #self._update_sizes(up=False) #really tree is gone
            self._remove_hook()
        elif self.hasAnyChild():
            noder = self.search(data)
            if noder:
                self._remove(noder)
            else:
                raise ValueError("No such data {} in tree".format(data))
        else:
            raise ValueError("No such data {} in tree".format(data))

    def _remove_hook(self, up=False, by=1):
        pass
    
    def _remove(self, node):
        if node.isLeaf():
            if node.isLeftChild():
                node.parent.left = None
            else:
                node.parent.right = None
            #node._update_sizes(up=False, by=node.count)
            node._remove_hook(by=node.count)
            del node
        elif node.hasBothChildren():
            s = node.successor()
            #successor is guaranteed to have right child only
            s.spliceOut()
            #s._update_sizes(up=False, by=s.count)
            s._remove_hook(by=s.count)
            #handled more generally above
            #s.right.parent = s.parent
            #s.parent.left = s.right
            node.data = s.data
            #diff = s.count - node.count            
            #node._update_sizes(by=diff)
            node._remove_hook(up=True, by = s.count - node.count)
            node.count = s.count
            del s #the node became the successor
        else: # one child case
            if node.hasLeftChild():
                if node.isLeftChild():
                    node.left.parent = node.parent
                    node.parent.left = node.left
                elif node.isRightChild():
                    node.left.parent = node.parent
                    node.parent.right = node.left
                else: #root
                    self.root = node.left
                #node._update_sizes(up=False, by=node.count)
                node._remove_hook(by=node.count)
                del node
            else:
                if node.isLeftChild():
                    node.right.parent = node.parent
                    node.parent.left = node.right
                elif node.isRightChild():
                    node.right.parent = node.parent
                    node.parent.right = node.right
                else: #root
                    self.root = node.right
                #node._update_sizes(up=False, by=node.count)
                node._remove_hook(by=node.count)
                del node
                    
    def findMin(self):
        minnode = self
        while minnode.hasLeftChild():
            minnode = minnode.left
        return minnode
    
    def findMax(self):
        maxnode = self
        while maxnode.hasRightChild():
            maxnode = maxnode.right
        return maxnode
    
    def successor(self):
        s = None
        if self.hasRightChild():
            s = self.right.findMin()
        else:
            if self.parent:
                if self.isLeftChild():
                    s = self.parent
                else:
                    self.parent.right=None
                    s = self.parent.successor()
                    self.parent.right=self
        return s
    
    def predecessor(self):
        p=None
        if self.hasLeftChild():
            p = self.left.findMax()
        else:
            if self.parent:
                if self.isRightChild():
                    p = self.parent
                else:
                    self.parent.left = None
                    p = self.parent.predecessor()
                    self.parent.left = self
        return p
            
    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                self.parent.left = None
            else:
                self.parent.right = None
        elif self.hasAnyChild():
            if self.hasLeftChild():
                if self.isLeftChild():
                    self.parent.left = self.left
                else:
                    self.parent.right = self.left
                self.left.parent = self.parent
            else:
                if self.isLeftChild():
                    self.parent.left = self.right
                else:
                    self.parent.right = self.right
                self.right.parent = self.parent
       

    #now implement various pythonic things
    def _rankof(self, data):
        if self.data == data:#found at top
            if self.hasLeftChild():
                return self.left.size + self.count, self.count
            else:
                return self.count, self.count
        elif data < self.data and self.left:
            return self.left._rankof(data)
        elif data > self.data and self.right:
            rtup = self.right._rankof(data)
            if self.hasLeftChild():
                return self.count + self.left.size+rtup[0], rtup[1]
            else:
                return self.count + rtup[0], rtup[1]
        else:
            raise ValueError("{} not found".format(x))
            
    def rankof(self, data):
        ranktup = self._rankof(data)
        return [ranktup[0] - e for e in range(ranktup[1])]    
    def __iter__(self):
        if self is not None:
            if self.hasLeftChild():
                for node in self.left:
                    yield node
            for _ in range(self.count):
                yield self
            if self.hasRightChild():
                for node in self.right:
                    yield node
                    
    def __len__(self):#expensive O(n) version
        start=0
        for node in self:
            start += 1
        return start
    
    def __getitem__(self, i):
        return self.ithorder(i+1)
    
    def __contains__(self, data):
        return self.search(data) is not None

## Q1.

Rank in an Augmented BST.

Given a key, what is its rank?

Write this in a method:

`def rankof(self, data):` which returns a list of the ranks for this key. Hint: it might be easy to get the largest rank for a key first, and then using the count for that key, return the list. Its possible to this in one traversal.

In [15]:
class AugmentedBinarySearchTree(BinarySearchTree):
        
    def __init__(self, data, parent=None):
        super().__init__(data, parent)
        self.size = 1
        
    def _update_sizes(self, up=True, by=1):
        if up:
            inc = by
        else:
            inc = -by
        self.size += inc
        curr = self
        while curr.parent is not None:
            curr.parent.size += inc
            curr = curr.parent
       
    def _insert_hook(self):#insert up, by 1
        self._update_sizes()
            
    def _remove_hook(self, up=False, by=1):
        self._update_sizes(up, by)
        
    
    def ithorder(self, i): #starts from 1
        if self.hasLeftChild():
            a = self.left.size
        else:
            a = 0
        dupes = self.count - 1
        if  a+1 <= i  < a+1 + dupes:
            return self
        if i < a + 1 : #wont go here for size 0 on left
            return self.left.ithorder(i)
        elif  a+1 <= i  <= a+1 + dupes:
            return self
        else:#ok to have self.right here and not check right child
            return self.right.ithorder(i - a -1 -dupes)
       
    #now implement various pythonic things
    
    def __len__(self):
        return self.size

    
    def __getitem__(self, i):
        return self.ithorder(i+1)
    
    #your code here


Check that you get the currect answer for both lists from the lecture

In [16]:
ourlist=[17,5,35,2,11,29,38,9,16,7,8]
duplist=[17,5,35,2,11,29,38,9,16,7,8,7,35,5,16,9,5]
def btree_fromlist(inputlist, cls=BinarySearchTree):
    start=inputlist[0]
    rest=inputlist[1:]
    btree = cls(start)
    for item in rest:
        btree.insert(item)
    return btree

In [17]:
btree = btree_fromlist(ourlist, AugmentedBinarySearchTree)
print([e.data for e in btree], len(btree))

[2, 5, 7, 8, 9, 11, 16, 17, 29, 35, 38] 11


In [18]:
# I am unable to plot the graph due to a problem of Jupyter in Windows
#plotbtree(btree, 'oninsertsaug', augmented=True, redo=False)

In [19]:
btreedup = btree_fromlist(duplist, AugmentedBinarySearchTree)
print([e.data for e in btreedup], len(btreedup), len(duplist))

[2, 5, 5, 5, 7, 7, 8, 9, 9, 11, 16, 16, 17, 29, 35, 35, 38] 17 17


In [20]:
#plotbtree(btreedup, 'oninsertsdupaug', augmented=True, redo=False)

In [21]:
for v in ourlist:
    print(v, btree.rankof(v))

17 [8]
5 [2]
35 [10]
2 [1]
11 [6]
29 [9]
38 [11]
9 [5]
16 [7]
7 [3]
8 [4]


In [22]:
for v in duplist:
    print(v, btreedup.rankof(v))

17 [13]
5 [4, 3, 2]
35 [16, 15]
2 [1]
11 [10]
29 [14]
38 [17]
9 [9, 8]
16 [12, 11]
7 [6, 5]
8 [7]
7 [6, 5]
35 [16, 15]
5 [4, 3, 2]
16 [12, 11]
9 [9, 8]
5 [4, 3, 2]


## Q2.

Implement a Key-Value Search true, which allows no duplicates, but rather, updates the value associated with the key. This will change how `insert` works. Inherit from the augmented tree:

`class KeyValueBinarySearchTree(AugmentedBinarySearchTree):`

- The constructor should look like this: `def __init__(self, key_value_tuple, parent=None):`. Pick the key and value out separately in the constructor, and initialize the super with just the key, setting an instance variable `self.value` to the value.
- insert wont duplicate any more, and `addLeftChild` and `addRightChild` will need to take the tuple in as they call the constructor for us.
- implement a `__getitem__`, `__setitem__`, and `__delitem__` so that you can use code like:

`btreekv['f']` for searching

`btreekv['f']=10` for inserting

In [85]:
#your code here
class KeyValueBinarySearchTree(AugmentedBinarySearchTree):
    def __init__(self, key_value_tuple, parent=None):
        data = key_value_tuple[0]
        value = key_value_tuple[1]
        super().__init__(data, parent)
        self.value = value
    
    def addLeftChild(self, key_value_tuple): 
        n = self.__class__(key_value_tuple, self)
        self.left = n
        return n
        
    def addRightChild(self, key_value_tuple):
        n = self.__class__(key_value_tuple, self)
        self.right = n
        return n

    def insert(self, key_value_tuple):
        data = key_value_tuple[0]
        value = key_value_tuple[1]
        if data < self.data:
            if self.hasLeftChild():
                self.left.insert(key_value_tuple)
            else:
                self.addLeftChild(key_value_tuple)
                self._insert_hook()
        elif data > self.data:
            if self.hasRightChild():
                self.right.insert(key_value_tuple)
            else:
                self.addRightChild(key_value_tuple)
                self._insert_hook()
        else: #duplicate value: update the value to the new one
            self.value = value
            self._insert_hook()
            
    def __getitem__(self, key):
        return self.search(key).value
    
    def __setitem__(self, key, item):
        self.insert((key,item))
        
    def __delitem(self, item):
        self._remove(item)

In [86]:
btreekv = KeyValueBinarySearchTree(('f', 3))

In [87]:
kvdata=zip(list('jeihrifhkdfks'), range(13))

In [88]:
for k,v in kvdata:
    btreekv[k]=v

In [89]:
[(e.data, e.value) for e in list(btreekv)]

[('d', 9),
 ('e', 1),
 ('f', 10),
 ('h', 7),
 ('i', 5),
 ('j', 0),
 ('k', 11),
 ('r', 4),
 ('s', 12)]

In [90]:
btreekv['f']

10

In [84]:
#plotbtree(btreekv, 'oninsertsdupaugkv', augmented=True, redo=False)

## Q3.

Use your code to make a third implementation of the `SimpleSet` interface from Monday.

In [147]:
class Env3():
    '''
    AbsFun: Uses a keyValue Binary Search Tree under the hood. 
    
    RepInv: only 1 value per key. replaces value for every new insert/update into the same key    
    
    '''

    def __init__(self):
        self._BST = KeyValueBinarySearchTree(('BinarySearchTreeRoot', None))
    
    #create an empty environment
    @classmethod
    def empty(cls):
        inst = cls()
        return inst        
    
    def extend_many(self, inputvars):
        for var in inputvars:
            self._BST[var] = inputvars[var]

    def extend(self, variable, value):
        self._BST[variable] = value
    
    def lookup(self, variable):
        if self._BST.search(variable) is not None and self._BST[variable] is not None:
            return (1, (variable, self._BST[variable]))
        else:
            raise NameError("{} not found in Environment".format(variable))

In [153]:
def one_env(envclass):
    "An environment with some Scheme standard procedures."
    import math, operator as op
    env = envclass.empty()
    env.extend_many(vars(math))
    env.extend_many({
        '+':op.add, '-':op.sub, '*':op.mul, '/':op.truediv, 
        'abs':     abs,
        'max':     max,
        'min':     min,
        'round':   round,
    })
    return env

globenv3 = one_env(Env3)

In [154]:
def typer(token):
    try:
        t = int(token)
        return t
    except ValueError:
        try:
            t = float(token)
            return t
        except ValueError:
            return str(token)
        
def lex(loc):
    tokenlist =  loc.replace('(', ' ( ').replace(')', ' ) ').split()
    return [typer(t) for t in tokenlist]

def syn(tokens):
    if len(tokens) == 0:
        return []
    token = tokens.pop(0)
    if token == '(':
        L = []
        while tokens[0] != ')':
            L.append(syn(tokens))
        tokens.pop(0) # pop off ')'
        return L
    else:
        if token==')':
            assert 1, "should not have got here"
        return token
    
def parse(loc):
    return syn(lex(loc))

def eval_ptree(x, env=globenv):
    print (x)
    if isinstance(x, str):      # variable lookup
        return env.lookup(x)
    elif not isinstance(x, list):  # constant
        return x
    elif len(x)==0: #noop
        return None 
    elif x[0] == 'let':         # variable definition
        (let, var, expression) = x
        #postorder traversal by nested eval is needed below
        # your code here
        env.extend(var, expression)
    else:
        operator_name = x[0]
        operator = env.lookup(operator_name)[1][1]
        if len(x) == 3:
            var1 = 0
            var2 = 0
            #print (x[1], x[2])
            if isinstance(x[1], (int, float, complex)):
                var1 = x[1]
            elif isinstance(x[1], list):
                var1 = eval_ptree(x[1])
            else:
                var1 = env.lookup(x[1])[1][1]

            if isinstance(x[2], (int, float, complex)):
                var2 = x[2]
            elif isinstance(x[2], list):
                var2 = eval_ptree(x[2])
            else:
                var2 = env.lookup(x[2])[1][1]
                
            #print (var1, var2)
            return operator(var1, var2)
        elif len(x) == 2:
            var1 = 0
            if isinstance(x[1], (int, float,  complex)):
                var1 = x[1]
            elif isinstance(x[1], list):
                var1 = eval_ptree(x[1])
            else:
                var1 = env.lookup(x[1])[1][1]

                
            return operator(var1)
        else:
            print ("3 operands not implemented yet")

In [155]:
class Program():
    
    def __init__(self, program, env):
        self.program = [e.strip() for e in program.split('\n')]
        self.env = env
        
    def __iter__(self):
        for line in self.program:
            yield line
    
    def parse(self):
        for l in iter(self):
            yield parse(l)
            
    def run(self):
        for l in iter(self):
            yield eval_ptree(parse(l), self.env)

In [156]:
program = """
(let radius 5)
(* pi (* radius radius))
"""

In [157]:
p2=Program(program, globenv)
for result in p2.run():
    print(result)

[]
None
['let', 'radius', 5]
None
['*', 'pi', ['*', 'radius', 'radius']]
['*', 'radius', 'radius']
78.53981633974483
[]
None
